In [55]:
import pyvista as pv
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


def vtp_to_threejs_blender(input_file, output_prefix):
    def write_obj(filename, vertices, faces):
        with open(filename, "w") as f:
            for v in vertices:
                f.write(f"v {v[0]} {v[1]} {v[2]}\n")
            for face in faces:
                f.write(f"f {face[0]+1} {face[1]+1} {face[2]+1}\n")

    # Read the .vtp file
    mesh = pv.read(input_file)
    vertices = mesh.points
    faces = mesh.faces.reshape(-1, 4)[:, 1:4]
    t_size = int(np.sqrt(len(faces)))
    if t_size**2 < len(faces):
        t_size = int(np.sqrt(len(faces))) + 1
        faces = np.append(
            faces, np.array([[0, 0, 0] for _ in range(t_size**2 - len(faces))]), axis=0
        )

    # Export geometry as .obj
    write_obj(f"{output_prefix}.obj", vertices, faces)

    # Get cell data (assuming scalar data is associated with cells)
    scalar_data = mesh.cell_data[list(mesh.cell_data.keys())[0]]

    # Normalize scalar data to [0, 1] range
    scalar_normalized = (scalar_data - np.min(scalar_data)) / (
        np.max(scalar_data) - np.min(scalar_data)
    )

    # Create a colormap
    cmap = plt.get_cmap("viridis")

    # Apply colormap to scalar data
    colors = (cmap(scalar_normalized)[:, :3] * 255).astype(np.uint8)

    # Create texture image
    # texture_size = 256  # Fixed size for texture
    texture_size = int(np.sqrt(len(faces)))  # Fixed size for texture
    texture = np.zeros((texture_size, texture_size, 3), dtype=np.uint8)
    print(faces.shape, texture.shape[0] * texture.shape[1], texture_size)

    # Map colors to texture
    for i, color in enumerate(colors):
        row = i // texture_size
        col = i % texture_size
        texture[row, col] = color

    # Save texture as image
    texture_image = Image.fromarray(texture)
    texture_image.save(f"{output_prefix}_texture.png")

    # Create material file (.mtl)
    with open(f"{output_prefix}.mtl", "w") as mtl_file:
        mtl_file.write("newmtl material0\n")
        mtl_file.write("Ka 1.000000 1.000000 1.000000\n")
        mtl_file.write("Kd 1.000000 1.000000 1.000000\n")
        mtl_file.write("Ks 0.000000 0.000000 0.000000\n")
        mtl_file.write("Tr 1.000000\n")
        mtl_file.write("illum 1\n")
        mtl_file.write(f"map_Kd {output_prefix}_texture.png\n")

    # Update .obj file to reference the material
    with open(f"{output_prefix}.obj", "r+") as obj_file:
        content = obj_file.read()
        obj_file.seek(0, 0)
        obj_file.write(f"mtllib {output_prefix}.mtl\n")
        obj_file.write("usemtl material0\n")
        obj_file.write(content)

    # Generate UV coordinates
    num_cells = len(scalar_data)
    uv_coords = np.zeros((num_cells * 3, 2))
    for i in range(num_cells):
        row = i // texture_size
        col = i % texture_size
        u = (col + 0.5) / texture_size
        v = 1 - (row + 0.5) / texture_size
        uv_coords[i * 3 : (i + 1) * 3] = [u, v]

    # Append UV coordinates to .obj file
    with open(f"{output_prefix}.obj", "a") as obj_file:
        for uv in uv_coords:
            obj_file.write(f"vt {uv[0]} {uv[1]}\n")

    print(
        f"Conversion complete. Output files: {output_prefix}.obj, {output_prefix}.mtl, {output_prefix}_texture.png"
    )


input_file = "./output/pressure/cp/default/cp.stats.vtp"
output_prefix = "./output/pressure/cp/default/v1"
vtp_to_threejs_blender(input_file, output_prefix)

(2916, 3) 2916 54
Conversion complete. Output files: ./output/pressure/cp/default/v1.obj, ./output/pressure/cp/default/v1.mtl, ./output/pressure/cp/default/v1_texture.png


In [69]:
import pyvista as pv
import numpy as np
from PIL import Image
import math
import matplotlib.pyplot as plt
import pyvista as pv
import numpy as np
from PIL import Image
import pathlib
import math


def create_texture_with_colormap(scalar_data, output_file, colormap_name="viridis"):
    num_triangles = len(scalar_data)

    # Find the smallest square that can contain all triangles
    texture_size = math.ceil(math.sqrt(num_triangles))

    # Create a square array, filling extra space with NaN
    square_data = np.full((texture_size * texture_size), np.nan)
    square_data[:num_triangles] = scalar_data
    # square_data = np.linspace(0, 1, (texture_size * texture_size))
    # square_data = square_data.reshape((texture_size, texture_size))

    # Normalize data
    norm = plt.Normalize(np.nanmin(square_data), np.nanmax(square_data))

    # Apply colormap
    cmap = plt.get_cmap(colormap_name)
    colored_data = cmap(norm(square_data))

    # Replace NaN with a specific color (e.g., transparent black)
    colored_data[np.isnan(square_data)] = [0, 0, 0, 0]

    # Convert to 8-bit color values
    colored_data = (colored_data * 255).astype(np.uint8)

    # Reshape and create image
    texture_data = colored_data.reshape((texture_size, texture_size, 4))
    img = Image.fromarray(texture_data[::-1], mode="RGBA")
    img.save(output_file)

    return texture_size


# def create_uv_mapping(num_triangles, texture_size):
#     uvs = []
#     for i in range(num_triangles):
#         u = (i % texture_size) / texture_size
#         v = (i // texture_size) / texture_size
#         # uvs.extend([u, v, u + 1 / texture_size, v, u, v + 1 / texture_size])
#         uvs.append([u, v, u + 1 / texture_size, v, u, v + 1 / texture_size])
#     uvs = np.array(uvs)
#     print(uvs.shape)
#     return uvs


# def write_obj_with_uv(filename, vertices, faces, uvs):
#     with open(filename, "w") as f:
#         f.write("mtllib ./output/pressure/cp/default/v1.mtl\n")
#         f.write("usemtl material0\n")
#         for v in vertices:
#             f.write(f"v {v[0]} {v[1]} {v[2]}\n")
#         for uv in uvs.reshape(-1, 2):
#             f.write(f"vt {uv[0]} {uv[1]}\n")
#         for i, face in enumerate(faces):
#             f.write(f"f {face[0]+1}/{3*i+1} {face[1]+1}/{3*i+2} {face[2]+1}/{3*i+3}\n")


def create_uv_mapping(num_triangles, texture_size):
    uvs = []
    for i in range(num_triangles):
        u = ((i + 0.5) % texture_size) / texture_size
        v = (i // texture_size + 0.5) / texture_size
        uvs.append([u, v])
    uvs = np.array(uvs)
    return uvs


def write_obj_with_uv(filename, vertices, faces, uvs):
    with open(filename, "w") as f:
        f.write("mtllib ./output/pressure/cp/default/v1.mtl\n")
        f.write("usemtl material0\n")
        for v in vertices:
            f.write(f"v {v[0]} {v[1]} {v[2]}\n")
        for uv in uvs:
            f.write(f"vt {uv[0]} {uv[1]}\n")
        for i, face in enumerate(faces):
            f.write(f"f {face[0]+1}/{i+1} {face[1]+1}/{i+1} {face[2]+1}/{i+1}\n")


# Usage
mesh_p = pathlib.Path("./output/pressure/cp/default/cp.stats.vtp")
mesh = pv.read(mesh_p)

vertices = mesh.points
faces = mesh.faces.reshape(-1, 4)[:, 1:4]  # Assuming triangular faces
scalar_data = mesh.cell_data["min"]

# Create texture with colormap
texture_size = create_texture_with_colormap(
    scalar_data, mesh_p.parent / "v1.png", colormap_name="viridis"
)

# Generate UV mapping
num_triangles = len(scalar_data)
uv_mapping = create_uv_mapping(num_triangles, texture_size)
print(uv_mapping)
# Write OBJ with UV coordinates
write_obj_with_uv(mesh_p.parent / "v1.obj", vertices, faces, uv_mapping)

print(f"Texture size: {texture_size}x{texture_size}")
print(f"Number of triangles: {num_triangles}")

[[0.00925926 0.00925926]
 [0.02777778 0.00925926]
 [0.0462963  0.00925926]
 ...
 [0.93518519 0.99074074]
 [0.9537037  0.99074074]
 [0.97222222 0.99074074]]
Texture size: 54x54
Number of triangles: 2915


In [57]:
import numpy as np
import pyvista as pv
from PIL import Image
import matplotlib.pyplot as plt

# Load the .vtp file
mesh = pv.read("path/to/your/file.vtp")

# Extract geometry and scalar data
points = mesh.points  # Geometry vertices (x, y, z)
cells = mesh.faces.reshape(-1, 4)[:, 1:]  # Cell connectivity (assuming triangular mesh)
scalar_data = mesh.cell_data["YourScalarDataName"]

mesh.save("output.obj")

# Assuming scalar data needs to be mapped as color
# Normalize scalar data
scalar_min = scalar_data.min()
scalar_max = scalar_data.max()
normalized_scalars = (scalar_data - scalar_min) / (scalar_max - scalar_min)

# Convert to RGB colors
cmap = plt.get_cmap("viridis")
colors = cmap(normalized_scalars)

# Create a texture image if scalar data represents colors
image_texture = np.zeros((mesh.n_cells, 3), dtype=np.uint8)
image_texture[:, :] = (colors[:, :3] * 255).astype(np.uint8)

# Export the texture (use an appropriate library like PIL)
img = Image.fromarray(image_texture)
img.save("output_texture.png")

FileNotFoundError: File (a:\Aerosim\Repositories\cfdmod\path\to\your\file.vtp) not found